In [1]:
%pip install "scikit-learn<1.6"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 75.7 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('/content/sample_data/KBO increased data_over 30 AB.csv')

In [3]:
R = df["R"].to_numpy()
R = R.reshape(-1, 1)

RBI = df["RBI"].to_numpy()
RBI = RBI.reshape(-1, 1)

WAR = df["WAR"].to_numpy()
WAR = WAR.reshape(-1, 1)

df = np.concatenate((R, RBI, WAR), axis=1)

# Split train set and test set
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

In [4]:
X_train = train_set[:, 0:2]
y_train = train_set[:, 2]
X_test = test_set[:, 0:2]
y_test = test_set[:, 2]

In [5]:
%pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.5 MB/s eta 0:00:00


In [6]:
%pip install lightgbm

In [7]:
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor


models = [
    ('linreg', LinearRegression()),
    ('ridge', Ridge(alpha=0.0001)),
    ('lasso', Lasso(alpha=0.0001)),
    ('elasticnet', ElasticNet(alpha=0.0001, l1_ratio=0.0001)),
    ('SVR', SVR(kernel='poly', C=0.012888951526111539, degree=3, epsilon=0.07498719388057506)),
    ('RandomForest', RandomForestRegressor(max_depth=6, n_estimators=500, random_state=42)),

    ('LGBM', LGBMRegressor(learning_rate=0.01, max_depth=6, n_estimators=31, random_state=42)),
    ('XGB', XGBRegressor(learning_rate=0.01, max_depth=6, n_estimators=31, random_state=42)),
    ('CatBoost', CatBoostRegressor(max_depth=10, n_estimators=61, random_state=42)),
    ('AdaBoost', AdaBoostRegressor(learning_rate=0.0020767704433677613, n_estimators=64, random_state=42)),
    ('GradientBoosting', GradientBoostingRegressor(learning_rate=0.0084608851019137, max_depth=16, n_estimators=91, random_state=42))
]

voting_reg = VotingRegressor(models)

pipeline = make_pipeline(StandardScaler(), voting_reg)
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000302 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 85
[LightGBM] [Info] Number of data points in the train set: 152, number of used features: 2
[LightGBM] [Info] Start training from score 1.139934
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('votingregressor',
                 VotingRegressor(estimators=[('linreg', LinearRegression()),
                                             ('ridge', Ridge(alpha=0.0001)),
                                             ('lasso', Lasso(alpha=0.0001)),
                                             ('elasticnet',
                                              ElasticNet(alpha=0.0001,
                                                         l1_ratio=0.0001)),
                                             ('SVR',
                                              SVR(C=0.012888951526111539,
                                                  epsilon=0.07498719388057506,
                                                  kernel='poly')),
                                             ('RandomForest',
                                              RandomFores...
                                                           n_jobs=None,
                                                           num_parallel_tree=None,
                                                           random_state=42, ...)),
                                             ('CatBoost',
                                              <catboost.core.CatBoostRegressor object at 0x79e55a791120>),
                                             ('AdaBoost',
                                              AdaBoostRegressor(learning_rate=0.0020767704433677613,
                                                                n_estimators=64,
                                                                random_state=42)),
                                             ('GradientBoosting',
                                              GradientBoostingRegressor(learning_rate=0.0084608851019137,
                                                                        max_depth=16,
                                                                        n_estimators=91,
                                                                        random_state=42))]))])

In [8]:
from sklearn.metrics import root_mean_squared_error
y_pred = pipeline.predict(X_train)
rmse = root_mean_squared_error(y_train, y_pred)
rmse

0.7735667928278751

In [9]:
y_pred_test = pipeline.predict(X_test)
rmse_test = root_mean_squared_error(y_test, y_pred_test)
rmse_test

0.672530306082241

In [10]:
# Hwang WAR(-0.30) Prediction
hwang = pipeline.predict([[60, 58]])
hwang

array([1.28784491])

In [11]:
# Rojas WAR(6.50) Prediction
Rojas = pipeline.predict([[108, 112]])
Rojas

array([4.66366123])

In [12]:
# Kim Do Yeong WAR(8.32) Prediction
DO_YEONG = pipeline.predict([[143, 109]])
DO_YEONG

array([6.14397605])

In [13]:
# Choi Jung WAR(4.55) Prediction
Choi_Jung = pipeline.predict([[93, 107]])
Choi_Jung

array([3.65229824])

In [14]:
# Kang Baek Ho WAR(2.12) Prediction
BaekHo = pipeline.predict([[92, 96]])
BaekHo

array([2.87936442])